In [3]:
# IMPORTS
import pandas as pd

import tkinter as tk
from tkinter import filedialog as fd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

import re
from unidecode import unidecode

from datetime import datetime

In [4]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')

# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']

# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [5]:
# funcion para ejecutar una script SQL
# estado = 0 para select
# estado = 1 para update
def dbQuery(q, estado = 0):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            if estado==1:
                conn.commit()  # Commit the transaction
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [9]:
# GENERAR CODIGO SLUG, CODIGO UNICO
def abbreviate_institution_name(name):
    # Eliminar acentos y otros caracteres especiales
    name = unidecode(name)
    # Mantener solo caracteres alfanuméricos, espacios y guiones
    name = re.sub(r"[^\w\s-]", "", name)
    # Convertir a minúsculas
    name = name.lower()
    # Reemplazar espacios y guiones por un solo guión
    abbreviated = re.sub(r"[\s-]+", "-", name)
    return abbreviated.strip('-')

In [7]:
# open file dialog
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".xlsx", ".xlsx")])
root.destroy()
file_name

'D:/OneDrive - Ingenio Azucarero Guabira S.A/APP CAÑERO/DATA PARA CARGAR/USUARIOS.xlsx'

In [8]:
data = pd.read_excel(file_name)
data

,CODIGO,NOMBRE,AP PATERNO,AP MATERNO,CORREO,TELF,NOM_USER,TIPO,INST
0,2710,JOSELITO,DEMIQUEL,JIMENEZ,jdemiquel@gmail.com,555555,jdemiquel,propietario,50
1,2751,TERESA,JIMENEZ,DE DEMIQUEL,tjimenez@gmail.com,555555,tjimenez,propietario,69
2,7572,ZENON,LAVERAN,VACA,zlaveran@gmail.com,555555,zlaveran,propietario,1
3,13172,ROSA,RODA,DE SORIOCO,rroda@gmail.com,555555,rroda,propietario,1
4,13456,BLANCA,JIMENEZ,DE TEJERINA,bjimenez@gmail.com,555555,bjimenez,propietario,1
5,42014,GUIDO OSCAR,TORRICO,LUIZAGA,gtorrico@gmail.com,555555,gtorrico,propietario,1
6,42042,WILSON JORGE,TORRICO,LUIZAGA,wtorrico@gmail.com,555555,wtorrico,propietario,1


In [12]:
fila = data.iloc[0]
fila['CODIGO']

2710

In [13]:
# optiene la fecha actual
now = datetime.now()
formatted_now = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_now)

2023-05-16 16:09:01
